## ESG Research 
### Yfinance, Pandas, NumPy


# Installation
* pip install pandas
* pip install numpy
* pip3 install tqdm
* pip3 install seaborn
* pip install yfinance | pip3 install yfinance
* pip install git+https://github.com/rodrigobercini/yfinance.git


In [2]:
#Import necessary libraries
import pandas as pd
import numpy as np
import yfinance as yf
from pandas import Series, DataFrame
from pandas.tseries import offsets
import csv
import datetime
import csv
from csv import writer
from tqdm import tqdm
import os
import time
from yahoofinancials import YahooFinancials
from sys import platform

dt = datetime.datetime.today()

YEAR = dt.year
MONTH = dt.month
CURRENT_CSV_FILE = bool

if platform == "linux" or platform == "linux2":
       op_sys = "linux"
elif platform == "darwin":
       op_sys = "Mac"
elif platform == "win32":
       op_sys = "Windows"
    
ESG_COLUMNS = {'palmOil', 'controversialWeapons', 'gambling', 'socialScore', 'nuclear',
       'furLeather', 'alcoholic', 'gmo', 'catholic', 'socialPercentile',
       'peerCount', 'governanceScore', 'environmentPercentile',
       'animalTesting', 'tobacco', 'totalEsg', 'highestControversy',
       'esgPerformance', 'coal', 'pesticides', 'adult', 'percentile',
       'peerGroup', 'smallArms', 'environmentScore', 'governancePercentile',
       'militaryContract', 
       #'Stock'
}

ALL_COLUMNS = {'palmOil', 'peerCount', 'environmentScore', 'militaryContract',
       'esgPerformance', 'coal', 'peerGroup', 'furLeather', 'gambling',
       'animalTesting', 'catholic', 'nuclear', 'totalEsg', 'adult',
       'environmentPercentile', 'highestControversy', 'socialScore',
       'percentile', 'alcoholic', 'socialPercentile', 'pesticides',
       'governancePercentile', 'controversialWeapons', 'gmo', 'smallArms',
       'tobacco', 'governanceScore', 'Stock Ticker', 'Sector', 'Name'
}

USER_PATH = '/Users/MichaelCalmette/Documents/Finance Research/ESG Data Files/'
DATE_PATH = "esg-{}-{}.csv".format(dt.month, dt.year)
#DATE_PATH = "esg-2-2022.csv"

FILE_PATH = os.path.join(USER_PATH,DATE_PATH)

Create functions to see if file exists / if there is data in there

In [2]:
f = open(FILE_PATH, 'w')
f.close
### Check if DF is empty

<function TextIOWrapper.close()>

In [3]:
def send_notification(title, t):
    if op_sys == "Mac":
        message = ("Time taken: {} minutes".format(t))
        command = f'''
        osascript -e 'display notification "{message}" with title "{title}"'
        '''
        os.system(command)
    

Need to compare data in esg-csv to all tickers

In [6]:
df = pd.read_csv('tickers.csv')
#all_data = pd.DataFrame(columns=ESG_COLUMNS)
# Only do if month is partially filled
all_data = pd.read_csv(FILE_PATH)
empty_stocks = pd.DataFrame(columns= ['Ticker'])

In [ ]:
tic = df.sample(n=1)
ticker_sym = tic.loc[tic.index[0],'Symbol'] # get ticker name ex: 'AAPL'
print(ticker_sym)
ticker = yf.Ticker(ticker_sym).history(period="1mo")
ticker

Yahoo Financials to get additional stock data

In [ ]:
yahoo_financials = YahooFinancials('AAPL')
print(yahoo_financials.get_summary_data())

Test pulling a random ticker and getting sustainability information

In [ ]:
tic = df.sample(n=1)
ticker_sym = tic.loc[tic.index[0],'Symbol'] # get ticker name ex: 'AAPL'
ticker = yf.Ticker(ticker_sym)
y = ticker.sustainability 
y

In [11]:
df1_transposed = pd.DataFrame()
no_data_stocks = 0
start_time = time.time()
temp = all_data[['Stock Ticker','Sector','Name']].copy()

for x in tqdm(range(10)):
    df_test = df.merge(temp, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only']
    tic = df_test.sample(n=1)
    ticker_sym = tic.loc[tic.index[0],'Symbol'] # get ticker name ex: 'AAPL'
    ticker = yf.Ticker(ticker_sym)
    y = ticker.sustainability 
    df1 = y

    if df1 is None:
        no_data_stocks += 1
        empty_stocks = empty_stocks.append({"Ticker": ticker_sym}, ignore_index = True)
    else:
        df1_transposed = df1.transpose()
        #print(df1_transposed.columns)
        df1_transposed['Stock Ticker'] = ticker_sym
        df1_transposed['Sector'] = tic.loc[tic.index[0],'Sector']
        df1_transposed['Name'] = tic.loc[tic.index[0],'Name']

        #all_data = df1_transposed
        all_data = all_data.append(df1_transposed, ignore_index = True)
    #temp = temp[temp['Stock Ticker'] != ticker_sym]

all_shape = len(all_data)
empty_shape = len(empty_stocks)
time_taken = round((time.time() - start_time) / 60,2)

print("There are: {} stocks left in the database".format(len(df_test)))
print("There are: {} stocks with no ESG data".format(no_data_stocks))

print("Dataframe shape: {}".format(all_shape))
print("Missing stocks: {}".format(empty_shape))

send_notification("Completed ESG Data Pull",time_taken)

100%|██████████| 10/10 [00:52<00:00,  5.21s/it]

There are: 66 stocks left in the database
There are: 10 stocks with no ESG data
Dataframe shape: 439
Missing stocks: 22


Make sure there are no duplicates

In [12]:
all_data

,adult,militaryContract,coal,socialPercentile,furLeather,environmentPercentile,socialScore,esgPerformance,environmentScore,totalEsg,...,gambling,alcoholic,governancePercentile,pesticides,palmOil,tobacco,smallArms,Stock Ticker,Sector,Name
0,False,True,False,None,False,None,15.04,LEAD_PERF,14.09,40.71,...,False,False,None,False,False,False,False,GE,Industrials,General Electric
1,False,True,False,None,False,None,7.53,AVG_PERF,9.25,23.82,...,False,False,None,False,False,False,False,ADI,Information Technology,Analog Devices
2,False,False,False,None,False,None,3.7,UNDER_PERF,3.23,13.02,...,False,False,None,False,False,False,False,DLR,Real Estate,Digital Realty Trust
3,False,False,False,None,False,None,14.93,AVG_PERF,0.33,22.47,...,False,False,None,False,False,False,False,BMY,Health Care,Bristol Myers Squibb
4,False,False,False,None,False,None,7.12,AVG_PERF,3.6,20.74,...,False,False,None,False,False,False,False,IDXX,Health Care,Idexx Laboratories
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434,False,False,False,None,False,None,9.36,AVG_PERF,8.44,24.33,...,False,False,None,False,False,False,False,SLB,Energy,Schlumberger
435,False,False,False,None,False,None,7.23,UNDER_PERF,3.83,16.07,...,False,False,None,False,False,False,False,ULTA,Consumer Discretionary,Ulta Beauty
436,False,True,False,None,False,None,9.95,AVG_PERF,10.63,29.12,...,False,False,None,False,False,False,False,HON,Industrials,Honeywell
437,False,False,False,None,False,None,17.22,AVG_PERF,0.15,25.05,...,False,False,None,False,False,False,False,VRTX,Health Care,Vertex Pharmaceuticals


In [31]:
s = len(all_data)
print(all_data.shape)
all_data = all_data.drop_duplicates()
dropped=len(all_data) - s
print("{} duplicates have been dropped".format(dropped))

(439, 30)
0 duplicates have been dropped


In [32]:
all_data.shape

(439, 30)

In [13]:
all_data['Stock Ticker'].value_counts()

WAB     1
INFO    1
FITB    1
BAC     1
JNPR    1
       ..
INTC    1
BBWI    1
PWR     1
NWSA    1
EL      1
Name: Stock Ticker, Length: 439, dtype: int64

In [33]:
if os.stat(FILE_PATH).st_size == 0:
    CURRENT_CSV_FILE = 0
    print('ESG File is empty')
    f = open(FILE_PATH, 'w')
    f.close

    with open(FILE_PATH, 'a', newline='') as f_object: 
        writer_object = writer(f_object) # Pass the CSV  file object to the writer() function
        writer_object.writerow(all_data) # Pass the data in the list as an argument into the writerow() function
        f_object.close()
    print('Column names have been entered.')

else:
    CURRENT_CSV_FILE = 1
    print('ESG File has column headers.')
    all_data.to_csv(FILE_PATH, mode='a', index=False, header=False)
    print('ESG Data has been entered.')

ESG File has column headers.
ESG Data has been entered.
